# Build the Neural Network

In [1]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

# Neural networks comprise of layers modules that perform operations on data

# The `torch.nn` namespace provides all the building blocks you need to build
# your own neural network
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [2]:
# We would like to use a hardware accelerator

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Define the Class

In [3]:
# We define our neural network by subclassing `nn.Module` and initialize the 
# neural network layers in __init__

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# To use the model, we pass it the input data. This executes the model's `forward`
# along with some background operations

# Calling the model on the input directly returns a 2-dimensional tensor with dim=0 
# corresponding to each output of 10 raw predicted values for each class, and dim=1
# corresponding to the individual values of each output
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Predicted class: tensor([8], device='cuda:0')


# Model Layers

In [4]:
# Take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it 
# through the network
input_image = torch.rand(3,28,28)
print(input_image.size())

# nn.Flatten

# We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array 
# of 784 pixel values
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

# nn.Linear

# The linear layer is a module that applies a linear transformation on the input using 
# its stored weights and biases
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

# nn.ReLU

# Non-linear activations are what creates the complex mappings between the model's inputs and outputs
# They are applied after linear transformations to introduce nonlinearity, helping neural networks
# learn a wide variety of phenomena
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

# nn.Sequential

# Is an ordered container of modules. The data is passed through all the modules in the same order
# as defined. You can use sequential containers to put together a quick network like seq_modules
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# nn.Softmax

# The last linear layer of the neural network returns logits - raw values in [-infty, infty]
# which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing
# the model's predicted probabilities for each class
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.3498, -0.2607, -0.2843, -0.3515,  0.0192, -0.0532, -0.0121, -0.2300,
         -0.0814, -0.0536,  0.1770,  0.3675,  0.4005,  0.1704,  0.0759,  0.1342,
         -0.1612, -1.0559,  0.0737,  0.5854],
        [ 0.1778, -0.2914, -0.1407,  0.0952,  0.0269,  0.2928,  0.4551, -0.6179,
         -0.0645, -0.0400,  0.0203,  0.0923,  0.5215,  0.2404,  0.1607,  0.1976,
          0.1241, -0.5954, -0.0700,  0.4129],
        [ 0.3001, -0.2829,  0.1042, -0.3024, -0.0787,  0.3321,  0.4076, -0.4719,
         -0.2150, -0.2098, -0.1827,  0.0698,  0.6719,  0.3080,  0.2433, -0.0816,
          0.2664, -0.6417, -0.0108,  0.8603]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3498, 0.0000, 0.0000, 0.0000, 0.0192, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1770, 0.3675, 0.4005, 0.1704, 0.0759, 0.1342, 0.0000, 0.0000,
         0.0737, 0.5854],
        [0.1778, 0.0000, 0.0000, 0.0952, 0.0269, 0.2928, 0.4551, 0.00

# Model Parameters

In [5]:
# Many layers inside a neural network are parameterized, have associated weights and biases that 
# are optimized during training.
# Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes
# all parameters accessible using your model's parameters() or named_parameters() methods

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0215, -0.0290, -0.0138,  ...,  0.0068, -0.0135,  0.0211],
        [ 0.0349, -0.0261,  0.0282,  ...,  0.0293, -0.0244, -0.0208]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0091,  0.0340], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0019,  0.0254,  0.0069,  ..., -0.0358,  0.0236, -0.0100],
        [-0.0383,  0.0002,  0.0161,  ...,  0.0394, -0.0165,  0.0050]],
       device='cuda:0', grad_fn=<Sl